# This notebook is trying to find the suburb with the highest growth using FOI COUNTS AS WELL

In [89]:
import pandas as pd
merged_df = pd.read_csv('/home/eeamanda/project-2-group-real-estate-industry-project-7-2025/Amanda-workspace/merged_df.csv')

In [90]:
merged_df.dtypes

Suburb                   object
date                     object
Median                    int64
Lat                     float64
Lng                     float64
SA2_CODE21                int64
SA2_NAME21               object
t                         int64
ERP_quarterly           float64
Income_quarterly_med    float64
dtype: object

Combine with FOI counts

In [91]:
foi_counts_df = pd.read_csv("/home/eeamanda/project-2-group-real-estate-industry-project-7-2025/Amanda-workspace/pivot_counts.csv")
foi_counts_df.dtypes

SA2_CODE21    int64
cultural      int64
education     int64
health        int64
others        int64
tourist       int64
dtype: object

In [92]:
# merge on SA2 code
merged_df = merged_df.merge(
    foi_counts_df,
    on='SA2_CODE21',
    how='left'   # keep all rows from merged_df
)
print(merged_df.head())


                                  Suburb        date  Median        Lat  \
0  Albert Park-Middle Park-West St Kilda  2017-03-01     520 -37.853484   
1  Albert Park-Middle Park-West St Kilda  2017-06-01     532 -37.853484   
2  Albert Park-Middle Park-West St Kilda  2017-09-01     530 -37.853484   
3  Albert Park-Middle Park-West St Kilda  2017-12-01     530 -37.853484   
4  Albert Park-Middle Park-West St Kilda  2018-03-01     550 -37.853484   

          Lng  SA2_CODE21   SA2_NAME21  t  ERP_quarterly  \
0  144.970161   206051128  Albert Park  0   16536.854795   
1  144.970161   206051128  Albert Park  1   16594.323288   
2  144.970161   206051128  Albert Park  2   16651.791781   
3  144.970161   206051128  Albert Park  3   16708.635616   
4  144.970161   206051128  Albert Park  4   16785.060274   

   Income_quarterly_med  cultural  education  health  others  tourist  
0          62618.808219         4          7       2     103       12  
1          62804.068493         4          7

In [93]:
merged_df

,Suburb,date,Median,Lat,Lng,SA2_CODE21,SA2_NAME21,t,ERP_quarterly,Income_quarterly_med,cultural,education,health,others,tourist
0,Albert Park-Middle Park-West St Kilda,2017-03-01,520,-37.853484,144.970161,206051128,Albert Park,0,16536.854795,62618.808219,4,7,2,103,12
1,Albert Park-Middle Park-West St Kilda,2017-06-01,532,-37.853484,144.970161,206051128,Albert Park,1,16594.323288,62804.068493,4,7,2,103,12
2,Albert Park-Middle Park-West St Kilda,2017-09-01,530,-37.853484,144.970161,206051128,Albert Park,2,16651.791781,62989.328767,4,7,2,103,12
3,Albert Park-Middle Park-West St Kilda,2017-12-01,530,-37.853484,144.970161,206051128,Albert Park,3,16708.635616,63172.575342,4,7,2,103,12
4,Albert Park-Middle Park-West St Kilda,2018-03-01,550,-37.853484,144.970161,206051128,Albert Park,4,16785.060274,63400.523288,4,7,2,103,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4813,Yarraville-Seddon,2024-03-01,570,-37.812809,144.884163,213031352,Yarraville,28,16280.121038,84590.600000,2,4,2,43,0
4814,Yarraville-Seddon,2024-06-01,590,-37.812809,144.884163,213031352,Yarraville,29,16337.039963,84590.600000,2,4,2,43,0
4815,Yarraville-Seddon,2024-09-01,595,-37.812809,144.884163,213031352,Yarraville,30,16393.958888,84590.600000,2,4,2,43,0
4816,Yarraville-Seddon,2024-12-01,600,-37.812809,144.884163,213031352,Yarraville,31,16450.259129,84590.600000,2,4,2,43,0


In [94]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# --- Stage 1: Within-suburb growth slopes ---
df = merged_df.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['Suburb','date'])

# numeric quarter index
t0 = df['date'].min()
df['t'] = ((df['date'] - t0) / pd.Timedelta(days=90)).astype(int)

train_mask = df['date'] <= '2024-09-01'
train_df = df.loc[train_mask]

growth_results = []
for suburb, df_train in train_df.groupby('Suburb'):
    if len(df_train) < 4:  # need enough quarters
        continue
    X_train = df_train[['t','ERP_quarterly','Income_quarterly_med']]
    y_train = df_train['Median']
    model = LinearRegression().fit(X_train, y_train)
    coef_t = model.coef_[0]   # slope wrt time
    growth_results.append({'Suburb': suburb, 'Growth_slope_t': coef_t})

growth_df = pd.DataFrame(growth_results)

# --- Stage 2: Cross-sectional regression on FOI counts ---
foi_df = (
    df.groupby('Suburb')
      .agg({
          'cultural':'first',
          'education':'first',
          'health':'first',
          'others':'first',
          'tourist':'first',
          'ERP_quarterly':'mean',
          'Income_quarterly_med':'mean'
      })
      .reset_index()
)

cross_df = growth_df.merge(foi_df, on='Suburb', how='left')

X = cross_df[['cultural','education','health','others','tourist',
              'ERP_quarterly','Income_quarterly_med']]
y = cross_df['Growth_slope_t']

X = sm.add_constant(X)
cross_model = sm.OLS(y, X).fit()
print(cross_model.summary())

# --- Stage 3: Predict long-run growth potential & rank ---
cross_df['Predicted_growth'] = cross_model.predict(X)

ranking = cross_df[['Suburb','Growth_slope_t','Predicted_growth',
                    'cultural','education','health','others','tourist',
                    'ERP_quarterly','Income_quarterly_med']] \
          .sort_values('Predicted_growth', ascending=False)

print("\n🏆 Top suburbs by predicted long-run growth:")
print(ranking.head(10))




                            OLS Regression Results                            
Dep. Variable:         Growth_slope_t   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     2.905
Date:                Wed, 08 Oct 2025   Prob (F-statistic):            0.00728
Time:                        17:54:47   Log-Likelihood:                -495.04
No. Observations:                 146   AIC:                             1006.
Df Residuals:                     138   BIC:                             1030.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -2.8801 

In [95]:
import statsmodels.api as sm

# attach FOI variables (they are constant per suburb, so take first row)
foi_df = (
    df.groupby('Suburb')
      .agg({
          'cultural':'first',
          'education':'first',
          'health':'first',
          'others':'first',
          'tourist':'first',
          'ERP_quarterly':'mean',
          'Income_quarterly_med':'mean'
      })
      .reset_index()
)

# merge with growth slopes
cross_df = growth_df.merge(foi_df, on='Suburb', how='left')

X = cross_df[['cultural','education','health','others','tourist',
              'ERP_quarterly','Income_quarterly_med']]
y = cross_df['Growth_slope_t']

X = sm.add_constant(X)
cross_model = sm.OLS(y, X).fit()
print(cross_model.summary())


                            OLS Regression Results                            
Dep. Variable:         Growth_slope_t   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     2.905
Date:                Wed, 08 Oct 2025   Prob (F-statistic):            0.00728
Time:                        17:54:47   Log-Likelihood:                -495.04
No. Observations:                 146   AIC:                             1006.
Df Residuals:                     138   BIC:                             1030.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -2.8801 

Merge with crime dataset

In [96]:
crime_df = pd.read_csv("/home/eeamanda/project-2-group-real-estate-industry-project-7-2025/Amanda-workspace/crime_dataset_weighted_to_SA2.csv")

In [97]:
crime_df

,SA2_CODE_2021,SA2_NAME_2021,Incidents_2016,Incidents_2017,Incidents_2018,Incidents_2019,Incidents_2020,Incidents_2021,Incidents_2022,Incidents_2023,...,VictimRate_2016,VictimRate_2017,VictimRate_2018,VictimRate_2019,VictimRate_2020,VictimRate_2021,VictimRate_2022,VictimRate_2023,VictimRate_2024,VictimRate_2025
0,201011001,Alfredton,14605.676027,14906.557159,14393.765122,13148.181941,14094.501630,11625.982235,12240.685623,13308.328350,...,5740.096618,5300.755085,4992.193889,4344.419848,4677.325451,3521.461799,3882.390463,3769.703537,4313.093763,4245.666569
1,201011002,Ballarat,14605.676027,14906.557159,14393.765122,13148.181941,14094.501630,11625.982235,12240.685623,13308.328350,...,5740.096618,5300.755085,4992.193889,4344.419848,4677.325451,3521.461799,3882.390463,3769.703537,4313.093763,4245.666569
2,201011005,Buninyong,14548.832253,14879.645181,14357.930442,13111.796323,14042.638700,11655.654556,12212.328264,13260.752451,...,5562.381238,5160.299900,4856.329142,4219.608124,4531.211146,3444.831718,3763.718188,3670.649736,4203.021792,4143.264308
3,201011006,Delacombe,14605.676027,14906.557159,14393.765122,13148.181941,14094.501630,11625.982235,12240.685623,13308.328350,...,5740.096618,5300.755085,4992.193889,4344.419848,4677.325451,3521.461799,3882.390463,3769.703537,4313.093763,4245.666569
4,201011007,Smythes Creek,2700.000000,2659.500000,2164.500000,2299.500000,2115.000000,2025.000000,1903.500000,2286.000000,...,1948.582849,1733.516119,1220.300979,1436.371273,1288.680949,1132.894103,1074.207180,1084.766775,1363.101340,1354.460609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,217031476,Otway,3221.500943,4307.789994,3312.247449,3523.099623,3416.339028,3272.212226,3226.838973,2935.916353,...,3473.457541,4149.108590,3381.753196,3480.510051,3034.928616,2604.330091,2671.411839,2527.837644,2600.896861,3151.186957
518,217041477,Moyne - East,1908.994094,2204.192887,2076.472650,2037.342973,2250.217049,2285.572393,2415.733116,1943.505238,...,1576.418926,2006.396908,1667.656375,1858.109515,1782.167501,1594.592720,1918.664676,1457.059579,1699.643378,1882.398689
519,217041478,Moyne - West,1922.715612,2207.259802,2094.322881,2068.248563,2276.036838,2298.159945,2439.588221,1965.221301,...,1565.732198,1997.477495,1658.048411,1867.486101,1779.233891,1589.659218,1915.735646,1453.411507,1686.998466,1873.358605
520,217041479,Warrnambool - North,4440.815176,5203.128393,5445.203123,6047.011239,4964.784621,4421.142158,4424.052573,4232.402245,...,3264.364425,3852.703839,3506.043496,4110.296172,3493.765722,3063.870347,3197.544185,2854.381338,3032.173712,3780.260863


interpolate crime df

In [98]:
import pandas as pd
import re
crime_df = crime_df.rename(columns={
    "SA2_CODE_2021": "SA2_CODE21",
    "SA2_NAME_2021": "SA2_NAME21"
})


def make_quarterly_crimerate(crime_df, start_year=2017, end_year=2025):
    """
    Convert wide annual CrimeRate_* columns to quarterly (QS-MAR) with linear interpolation.
    Keeps SA2_CODE21 and SA2_NAME21.
    """

    # --- 1) Detect CrimeRate columns
    cr_cols = [c for c in crime_df.columns if re.fullmatch(r"CrimeRate_\d{4}", c)]
    if not cr_cols:
        raise ValueError("No CrimeRate_* columns found in dataframe.")

    # --- 2) Melt to long annual format ---
    long_df = crime_df.melt(
        id_vars=["SA2_CODE21", "SA2_NAME21"],
        value_vars=cr_cols,
        var_name="year_lbl",
        value_name="CrimeRate"
    )

    # Extract numeric year
    long_df["year"] = long_df["year_lbl"].str.extract(r"(\d{4})").astype(int)
    long_df = long_df.drop(columns="year_lbl")

    # Collapse duplicates (just in case)
    long_df = long_df.groupby(["SA2_CODE21", "SA2_NAME21", "year"], as_index=False)["CrimeRate"].mean()

    # Add annual date anchors (Jan 1 of each year)
    long_df["date"] = pd.to_datetime(long_df["year"].astype(str) + "-01-01")

    # --- 3) Build clean quarterly index (3rd of March cycle) ---
    q_start = pd.Timestamp(f"{start_year}-03-01")
    q_end   = pd.Timestamp(f"{end_year}-12-01")
    quarterly_index = pd.date_range(start=q_start, end=q_end, freq="QS-MAR")

    # --- 4) Interpolate per SA2 ---
    out = []
    for code, g in long_df.groupby("SA2_CODE21", sort=False):
        g = g.sort_values("date").set_index("date")[["CrimeRate"]]

        # ensure no duplicates
        g = g[~g.index.duplicated(keep="last")]

        # union annual+quarterly, then interpolate linearly
        union_idx = g.index.union(quarterly_index)
        g_u = g.reindex(union_idx)
        g_u["CrimeRate"] = g_u["CrimeRate"].interpolate(method="time", limit_direction="both")

        # keep only quarterly grid (3rd of Mar/Jun/Sep/Dec)
        g_q = g_u.reindex(quarterly_index)

        # attach code + name
        name = long_df.loc[long_df["SA2_CODE21"] == code, "SA2_NAME21"].iloc[0]
        g_q = g_q.reset_index().rename(columns={"index": "date", "CrimeRate": "CrimeRate_quarterly"})
        g_q["SA2_CODE21"] = code
        g_q["SA2_NAME21"] = name

        out.append(g_q)

    # --- 5) Final tidy result ---
    crime_quarterly = pd.concat(out, ignore_index=True)
    crime_quarterly = crime_quarterly[["SA2_CODE21", "SA2_NAME21", "date", "CrimeRate_quarterly"]]

    return crime_quarterly



In [99]:
crime_quarterly = make_quarterly_crimerate(crime_df)
print(crime_quarterly.head(8))

   SA2_CODE21 SA2_NAME21       date  CrimeRate_quarterly
0   201011001  Alfredton 2017-03-01          8655.782221
1   201011001  Alfredton 2017-06-01          8539.465274
2   201011001  Alfredton 2017-09-01          8423.148327
3   201011001  Alfredton 2017-12-01          8308.095694
4   201011001  Alfredton 2018-03-01          8129.155817
5   201011001  Alfredton 2018-06-01          7911.246608
6   201011001  Alfredton 2018-09-01          7693.337398
7   201011001  Alfredton 2018-12-01          7477.796767


In [100]:
# --- 1) Ensure date columns are datetime ---
merged_df["date"] = pd.to_datetime(merged_df["date"])


# --- 3) Merge on SA2_CODE21 + date ---
merged_df = pd.merge(
    merged_df,
    crime_quarterly[["SA2_CODE21", "date", "CrimeRate_quarterly"]],
    on=["SA2_CODE21", "date"],
    how="left"   # keeps all property rows even if crime is missing
)

# --- 4) Optional: check merge coverage ---
matched = merged_df["CrimeRate_quarterly"].notna().mean() * 100
print(f"✅ Merge complete — {matched:.1f}% of property rows have matched crime rates.")


✅ Merge complete — 100.0% of property rows have matched crime rates.


In [101]:
merged_df

,Suburb,date,Median,Lat,Lng,SA2_CODE21,SA2_NAME21,t,ERP_quarterly,Income_quarterly_med,cultural,education,health,others,tourist,CrimeRate_quarterly
0,Albert Park-Middle Park-West St Kilda,2017-03-01,520,-37.853484,144.970161,206051128,Albert Park,0,16536.854795,62618.808219,4,7,2,103,12,9454.822696
1,Albert Park-Middle Park-West St Kilda,2017-06-01,532,-37.853484,144.970161,206051128,Albert Park,1,16594.323288,62804.068493,4,7,2,103,12,9270.029105
2,Albert Park-Middle Park-West St Kilda,2017-09-01,530,-37.853484,144.970161,206051128,Albert Park,2,16651.791781,62989.328767,4,7,2,103,12,9085.235514
3,Albert Park-Middle Park-West St Kilda,2017-12-01,530,-37.853484,144.970161,206051128,Albert Park,3,16708.635616,63172.575342,4,7,2,103,12,8902.450549
4,Albert Park-Middle Park-West St Kilda,2018-03-01,550,-37.853484,144.970161,206051128,Albert Park,4,16785.060274,63400.523288,4,7,2,103,12,8809.998957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4813,Yarraville-Seddon,2024-03-01,570,-37.812809,144.884163,213031352,Yarraville,28,16280.121038,84590.600000,2,4,2,43,0,8877.119039
4814,Yarraville-Seddon,2024-06-01,590,-37.812809,144.884163,213031352,Yarraville,29,16337.039963,84590.600000,2,4,2,43,0,9304.615102
4815,Yarraville-Seddon,2024-09-01,595,-37.812809,144.884163,213031352,Yarraville,30,16393.958888,84590.600000,2,4,2,43,0,9732.111164
4816,Yarraville-Seddon,2024-12-01,600,-37.812809,144.884163,213031352,Yarraville,31,16450.259129,84590.600000,2,4,2,43,0,10154.960531


In [102]:
n_stops_df = pd.read_csv("/home/eeamanda/project-2-group-real-estate-industry-project-7-2025/Amanda-workspace/SA2_with_ptstops.csv")
n_stops_df
n_stops_df["SA2_CODE21"] = pd.to_numeric(n_stops_df["SA2_CODE21"], errors="coerce").astype("Int64")

In [110]:
merged_df = merged_df.merge(
    n_stops_df,
    on=["SA2_CODE21"],
    how="left",
    indicator=True  # adds a "_merge" column showing match status
)

# --- 3️⃣ Check match summary ---
print("🔍 Merge summary:")
print(merged_df["_merge"].value_counts())

matched = (merged_df["_merge"] == "both").mean() * 100
print(f"✅ {matched:.1f}% of rows merged successfully.")

# Drop the right-side name
merged_df = merged_df.drop(columns=["SA2_NAME21_y","_merge"])

# Rename the left-side name back to SA2_NAME21
merged_df = merged_df.rename(columns={"SA2_NAME21_x": "SA2_NAME21"})

merged_df

🔍 Merge summary:
_merge
both          36333
left_only        33
right_only        0
Name: count, dtype: int64
✅ 99.9% of rows merged successfully.


,Suburb,date,Median,Lat,Lng,SA2_CODE21,SA2_NAME21,t,ERP_quarterly,Income_quarterly_med,cultural,education,health,others,tourist,CrimeRate_quarterly,n_pt_stops_suburb,n_train_stops_suburb,n_tram_stops_suburb
0,Albert Park-Middle Park-West St Kilda,2017-03-01,520,-37.853484,144.970161,206051128,Albert Park,0,16536.854795,62618.808219,4,7,2,103,12,9454.822696,62.0,21.0,41.0
1,Albert Park-Middle Park-West St Kilda,2017-03-01,520,-37.853484,144.970161,206051128,Albert Park,0,16536.854795,62618.808219,4,7,2,103,12,9454.822696,62.0,21.0,41.0
2,Albert Park-Middle Park-West St Kilda,2017-03-01,520,-37.853484,144.970161,206051128,Albert Park,0,16536.854795,62618.808219,4,7,2,103,12,9454.822696,62.0,21.0,41.0
3,Albert Park-Middle Park-West St Kilda,2017-03-01,520,-37.853484,144.970161,206051128,Albert Park,0,16536.854795,62618.808219,4,7,2,103,12,9454.822696,62.0,21.0,41.0
4,Albert Park-Middle Park-West St Kilda,2017-03-01,520,-37.853484,144.970161,206051128,Albert Park,0,16536.854795,62618.808219,4,7,2,103,12,9454.822696,62.0,21.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36361,Yarraville-Seddon,2025-03-01,600,-37.812809,144.884163,213031352,Yarraville,32,16469.438332,84590.600000,2,4,2,43,0,10299.008117,90.0,90.0,0.0
36362,Yarraville-Seddon,2025-03-01,600,-37.812809,144.884163,213031352,Yarraville,32,16469.438332,84590.600000,2,4,2,43,0,10299.008117,90.0,90.0,0.0
36363,Yarraville-Seddon,2025-03-01,600,-37.812809,144.884163,213031352,Yarraville,32,16469.438332,84590.600000,2,4,2,43,0,10299.008117,90.0,90.0,0.0
36364,Yarraville-Seddon,2025-03-01,600,-37.812809,144.884163,213031352,Yarraville,32,16469.438332,84590.600000,2,4,2,43,0,10299.008117,90.0,90.0,0.0


In [113]:
# Filter rows with at least one NaN
nan_rows = merged_df[merged_df.isna().any(axis=1)]

print("🔍 Rows with NaN values in merged_df:")
print(nan_rows.head(20))   # show first 20
print(f"\nTotal rows with NaN: {len(nan_rows)}")

#Put 0 for nans
merged_df = merged_df.fillna(0)



🔍 Rows with NaN values in merged_df:
            Suburb       date  Median        Lat         Lng  SA2_CODE21  \
33363  Wanagaratta 2017-03-01     260 -36.358891  146.309658   204021066   
33364  Wanagaratta 2017-06-01     265 -36.358891  146.309658   204021066   
33365  Wanagaratta 2017-09-01     270 -36.358891  146.309658   204021066   
33366  Wanagaratta 2017-12-01     270 -36.358891  146.309658   204021066   
33367  Wanagaratta 2018-03-01     270 -36.358891  146.309658   204021066   
33368  Wanagaratta 2018-06-01     270 -36.358891  146.309658   204021066   
33369  Wanagaratta 2018-09-01     277 -36.358891  146.309658   204021066   
33370  Wanagaratta 2018-12-01     280 -36.358891  146.309658   204021066   
33371  Wanagaratta 2019-03-01     280 -36.358891  146.309658   204021066   
33372  Wanagaratta 2019-06-01     290 -36.358891  146.309658   204021066   
33373  Wanagaratta 2019-09-01     290 -36.358891  146.309658   204021066   
33374  Wanagaratta 2019-12-01     290 -36.358891  1

In [124]:
import statsmodels.api as sm

X = merged_df[[
    "ERP_quarterly", "Income_quarterly_med", "CrimeRate_quarterly",
    "cultural", "education", "health", "others", "tourist",
    "n_pt_stops_suburb", "n_train_stops_suburb", "n_tram_stops_suburb","t"
]]
y = merged_df["Median"]

X = sm.add_constant(X)  # adds intercept
model_ols = sm.OLS(y, X).fit()
print(model_ols.summary())


                            OLS Regression Results                            
Dep. Variable:                 Median   R-squared:                       0.560
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     3857.
Date:                Wed, 08 Oct 2025   Prob (F-statistic):               0.00
Time:                        18:27:07   Log-Likelihood:            -2.0163e+05
No. Observations:               36366   AIC:                         4.033e+05
Df Residuals:                   36353   BIC:                         4.034e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  208.9695 

In [126]:
import pandas as pd
import numpy as np
from linearmodels.panel import RandomEffects

# 1) Prep
df = merged_df.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['Suburb', 'date'])

# numeric quarter index (trend)
t0 = df['date'].min()
df['t'] = ((df['date'] - t0) / pd.Timedelta(days=90)).astype(int)

# 2) Choose features
time_varying = ["ERP_quarterly", "Income_quarterly_med", "CrimeRate_quarterly", "t"]
time_invariant = [
    "cultural", "education", "health", "others", "tourist",
    "n_pt_stops_suburb", "n_train_stops_suburb", "n_tram_stops_suburb"
]

# keep only columns that exist (safety)
time_varying = [c for c in time_varying if c in df.columns]
time_invariant = [c for c in time_invariant if c in df.columns]

# 3) Mundlak: add suburb means of time-varying regressors
means = (
    df.groupby("Suburb")[time_varying]
      .transform("mean")
      .add_suffix("_mean_suburb")
)
df = pd.concat([df, means], axis=1)

# 4) Build panel index
panel = df.set_index(["Suburb", "date"])

y = panel["Median"]
X_cols = time_varying + time_invariant + [c + "_mean_suburb" for c in time_varying]
X = panel[X_cols]

# Optionally drop rows with missing values in used columns
valid = X.notna().all(axis=1) & y.notna()
y = y[valid]
X = X[valid]

# 5) Fit Random Effects (Mundlak)
re_mundlak = RandomEffects(y, X)
res = re_mundlak.fit(cov_type="robust")  # heteroskedasticity-robust SEs

print(res.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:                 Median   R-squared:                        0.7560
Estimator:              RandomEffects   R-squared (Between):              0.5227
No. Observations:               36366   R-squared (Within):               0.7560
Date:                Wed, Oct 08 2025   R-squared (Overall):              0.6040
Time:                        18:27:51   Log-likelihood                -1.713e+05
Cov. Estimator:                Robust                                           
                                        F-statistic:                      7508.7
Entities:                         146   P-value                           0.0000
Avg Obs:                       249.08   Distribution:                F(15,36350)
Min Obs:                       33.000                                           
Max Obs:                       693.00   F-statistic (robust):             7565.4
                            

In [130]:
import statsmodels.formula.api as smf

df = merged_df.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['Suburb','date']).reset_index(drop=True)

# numeric quarter index
t0 = df['date'].min()
df['t'] = ((df['date'] - t0) / pd.Timedelta(days=90)).astype(int)

# log-transform for stability
df['logMedian'] = np.log(df['Median'].clip(lower=1))

# Mixed effects: random intercept + random slope on t per suburb
# You can also allow slopes on ERP or Income if you want suburb-specific elasticities
formula = (
    "logMedian ~ ERP_quarterly + Income_quarterly_med + CrimeRate_quarterly "
    "+ cultural + education + health + others + tourist "
    "+ n_pt_stops_suburb + n_train_stops_suburb + n_tram_stops_suburb + t"
)

md = smf.mixedlm(formula, df, groups=df["Suburb"], re_formula="~ t")
mres = md.fit(method="lbfgs")
print(mres.summary())

import numpy as np

# Predicted values
yhat = mres.fittedvalues
y = df["logMedian"]

# Residual sum of squares
ss_res = np.sum((y - yhat)**2)

# Total sum of squares (relative to mean of y)
ss_tot = np.sum((y - np.mean(y))**2)

r2 = 1 - ss_res/ss_tot
print(f"Pseudo R²: {r2:.4f}")




/home/eeamanda/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/eeamanda/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


             Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   logMedian 
No. Observations:     36366     Method:               REML      
No. Groups:           146       Scale:                0.0022    
Min. group size:      33        Log-Likelihood:       58608.9190
Max. group size:      693       Converged:            Yes       
Mean group size:      249.1                                     
----------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z| [0.025 0.975]
----------------------------------------------------------------
Intercept             5.234    0.049 107.862 0.000  5.139  5.329
ERP_quarterly         0.000    0.000 114.391 0.000  0.000  0.000
Income_quarterly_med -0.000    0.000 -24.286 0.000 -0.000 -0.000
CrimeRate_quarterly   0.000    0.000  68.104 0.000  0.000  0.000
cultural             -0.015    0.006  -2.508 0.012 -0.027 -0.003
education            -0.042    0.006  -